Code adapted from https://gogul09.github.io/software/flower-recognition-deep-learning

In [16]:
# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.models import model_from_json

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import pickle
import datetime
import time
import tensorflow as tf

from keras import backend as K

sess = tf.Session()
K.set_session(sess)

import keras 
tbCallBack = keras.callbacks.TensorBoard(log_dir='H:/output/inceptionv3/Logs', write_graph=True)


In [21]:
# config variables
#model_name = config["model"]
#weights = config["weights"]
#include_top = config["include_top"]
#train_path = config["train_path"]
#features_path = config["features_path"]
#labels_path = config["labels_path"]
#test_size = config["test_size"]
#results = config["results"]
#model_path = config["model_path"]
#weights_path = config["weights_path"]

model_name = "vgg16"
weights = "imagenet"
include_top = False

train_path= "H:/Output2/training"
val_path= "H:/Output2/test"

features_path= "H:/Output2/meta/features.h5"
Labels_path = "H:/Output2/meta/labels.h5"

Val_features_path= "H:/Output2/meta/valfeatures.h5"
Val_Labels_path = "H:/Output2/meta/vallabels.h5"

results = "H:/Output2/meta/results.txt"
classifier_path = "classifier.cpickle"
model_path = "H:/Output2/meta/model"
weights_path = "H:/Output2/meta/weight"

test_size= 0.20
seed = 9
num_classes = 101




In [22]:
%%time
# create the pretrained models
# check for pretrained weight usage or not
# check for top layers to be included or not
if model_name == "vgg16":
    base_model = VGG16(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    image_size = (224, 224)
elif model_name == "vgg19":
    base_model = VGG19(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    image_size = (224, 224)
elif model_name == "resnet50":
    base_model = ResNet50(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('flatten').output)
    image_size = (224, 224)
elif model_name == "inceptionv3":
    base_model = InceptionV3(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)
    image_size = (299, 299)
elif model_name == "xception":
    base_model = Xception(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)
    image_size = (299, 299)
else:
    base_model = None

print ("[INFO] successfully loaded base model and model...")


[INFO] successfully loaded base model and model...
Wall time: 3.33 s


C:\ProgramData\Anaconda3\envs\trabalho\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  


In [23]:
%%time
# path to training dataset
train_labels = os.listdir(train_path)

# encode the labels
print("[INFO] encoding labels...")
le = LabelEncoder()
le.fit([tl for tl in train_labels])

# variables to hold features and labels
features = []
labels   = []



[INFO] encoding labels...
Wall time: 3 ms


In [ ]:
train_labels = os.listdir(train_path)
print(train_labels)

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_sou

In [ ]:
%%time
# loop over all the labels in the folder
for i, label in enumerate(train_labels):
    cur_path = train_path + "/" + label
    a=0
    for image_path in glob.glob(cur_path + "/*.jpg"):
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x)
        flat = feature.flatten()
        features.append(flat)
        labels.append(label)
        print ("[INFO] processed ",i,a)
        a=a+1
    print ("[INFO] completed label",label)
    

# encode the labels using LabelEncoder
targetNames = np.unique(labels)
le = LabelEncoder()
le_labels = le.fit_transform(labels)

print (le_labels)

[INFO] processed  0 0
[INFO] processed  0 1
[INFO] processed  0 2
[INFO] processed  0 3
[INFO] processed  0 4
[INFO] processed  0 5
[INFO] processed  0 6
[INFO] processed  0 7
[INFO] processed  0 8
[INFO] processed  0 9
[INFO] processed  0 10
[INFO] processed  0 11
[INFO] processed  0 12
[INFO] processed  0 13
[INFO] processed  0 14
[INFO] processed  0 15
[INFO] processed  0 16
[INFO] processed  0 17
[INFO] processed  0 18
[INFO] processed  0 19
[INFO] processed  0 20
[INFO] processed  0 21
[INFO] processed  0 22
[INFO] processed  0 23
[INFO] processed  0 24
[INFO] processed  0 25
[INFO] processed  0 26
[INFO] processed  0 27
[INFO] processed  0 28
[INFO] processed  0 29
[INFO] processed  0 30
[INFO] processed  0 31
[INFO] processed  0 32
[INFO] processed  0 33
[INFO] processed  0 34
[INFO] processed  0 35
[INFO] processed  0 36
[INFO] processed  0 37
[INFO] processed  0 38
[INFO] processed  0 39
[INFO] processed  0 40
[INFO] processed  0 41
[INFO] processed  0 42
[INFO] processed  0 4

In [12]:
%%time
# path to training dataset
val_labels = os.listdir(val_path)

# encode the labels
print("[INFO] encoding labels...")
val_le = LabelEncoder()
val_le.fit([vl for vl in val_labels])

# variables to hold features and labels
valfeatures = []
vallabels   = []



[INFO] encoding labels...
Wall time: 1 ms


In [13]:
%%time
# loop over all the labels in the folder
for i, label in enumerate(val_labels):
    cur_path = val_path + "/" + label
    a=0
    for image_path in glob.glob(cur_path + "/*.jpg"):
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        valfeature = model.predict(x)
        flat = valfeature.flatten()
        valfeatures.append(flat)
        vallabels.append(label)
        print ("[INFO] processed ",i,a)
        a=a+1
    print ("[INFO] completed label",label)
    

# encode the labels using LabelEncoder
ValtargetNames = np.unique(vallabels)
valle = LabelEncoder()
valle_labels = valle.fit_transform(vallabels)

print (valle_labels)

NameError: name 'image_size' is not defined

In [14]:
# get the shape of training labels
print ("[STATUS] training labels:",le_labels)
print ("[STATUS] training labels shape:",le_labels.shape)

# save features and labels
h5f_data = h5py.File(features_path, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File(Labels_path, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

NameError: name 'le_labels' is not defined

In [ ]:
# get the shape of training labels
print ("[STATUS] training labels:",valle_labels)
print ("[STATUS] training labels shape:",valle_labels.shape)

# save features and labels
h5f_val_data = h5py.File(Val_features_path, 'w')
h5f_val_data.create_dataset('dataset_1', data=np.array(valfeatures))

h5f_val_label = h5py.File(Val_Labels_path, 'w')
h5f_val_label.create_dataset('dataset_1', data=np.array(valle_labels))

h5f_val_data.close()
h5f_val_label.close()

In [ ]:
%%time
# save model and weights
model_json = model.to_json()



In [ ]:
%%time
#with open(model_path + str(test_size) + ".json", "w") as json_file:
with open("H:/Output2/meta/model0.1.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
%%time
# save weights
#model.save_weights(model_path + str(test_size) + ".h5")
model.save_weights("H:/output/inceptionv3/model0.1.h5")
print("[STATUS] saved model and weights to disk..")

print ("[STATUS] features and labels saved..")


